# Run Registry Data
This notebook showcases the use of the Run Registry api, `runregistry`, and serves as an example of how to use it in order to programmatically produce a RR json. You can also product this json through the website by following this link:
https://cmsrunregistry.web.cern.ch/json_portal

Read the documentation found in the `runregistry_api_client` GitHub repository for more information on how to authenticate: https://github.com/cms-DQM/runregistry_api_client

In [ ]:
import runregistry
import itertools
runregistry.setup("production")

In [ ]:
# Getting run registry data example
filter = {
    "run_number": {
        # "and": [{">=": 365753}, {"<=": 373710}]
        "and": [{"=": 365753}]
    },
    "class": {
        "and": [{"=": "Collisions23"}]
    },
    "tracker-strip": "GOOD",
    "tracker-pixel": "GOOD",
}

runs = runregistry.get_runs(filter=filter)
runs

In [ ]:
# Generating json
json_logic = {
  "and": [
    {"==": [{"var": "lumisection.rr.tracker-pixel"}, "GOOD"        ]},
    {"==": [{"var": "lumisection.rr.tracker-strip"}, "GOOD"        ]},
    {"==": [{"var": "lumisection.rr.tracker-track"}, "GOOD"        ]},
  ]
}

generated_jsons = {}
datasets = ["/Express/Collisions{}/DQM"]
# years = ["2022", "2023", "2024"]
years = ["2022"]
for dataset, year in itertools.product(datasets, years):
    dataset_name = dataset.format(year)
    print(dataset_name) 
    generated_jsons[dataset_name] = {
        **runregistry.create_json(
            json_logic=json_logic, 
            dataset_name_filter=dataset.format(year)
        )["generated_json"]
    }

In [ ]:
jsons = {}
for key, val in generated_jsons.items():
    jsons = {**jsons, **val}
jsons

In [ ]:
import json
import os

json_fname = "Express-Collisions-2022_pixel-strip-track-good.json"
outfile_path = "../jsons"

with open (os.path.join(outfile_path, json_fname), "w") as outfile:
    json.dump(jsons, outfile)